In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import random
from heapq import heappop, heappush

In [2]:
'''Grid'''
class Map:

    # 
    def __init__(self):
        '''
        Default constructor
        '''

        self.width = 0
        self.height = 0
        self.cells = []
    

    def SetGridCells(self, width, height, gridCells):
        '''
        Initialization of map by list of cells.
        '''
        self.width = width
        self.height = height
        self.cells = gridCells


    def inBounds(self, i, j):
        '''
        Check if the cell is on a grid.
        '''
        return (0 <= i < self.width) and (0 <= j < self.height)
    

    def Traversable(self, i, j):
        '''
        Check if the cell is not an obstacle.
        '''
        return not self.cells[j][i]


    def GetNeighbors(self, i, j):
        '''
        Get a list of neighbouring cells as (i,j) tuples.
        '''   
        neighbors = [[i, j]]
        delta = [[0, 1], [1, 0], [0, -1], [-1, 0]]

        for d in delta:
            if self.inBounds(i + d[0], j + d[1]) and self.Traversable(i + d[0], j + d[1]):
                neighbors.append((i + d[0], j + d[1]))
        return neighbors

In [3]:
def read_map_from_movingai_file(path):
    map_file = open(path)
    count = 0
    name = map_file.readline()
    height = int(map_file.readline().split()[1])
    width = int(map_file.readline().split()[1])
    type_ = map_file.readline()
    cells = [[0 for _ in range(width)] for _ in range(height)]
    
    i = 0
    j = 0

    for l in map_file:
        j = 0
        for c in l:
            if c == '.':
                cells[i][j] = 0
            elif c == 'T' or c == '@':
                cells[i][j] = 1
            else:
                continue
            
            j += 1
            
        if j != width:
            raise Exception("Size Error. Map width = ", j, ", but must be", width, "(map line: ", i, ")")
                
        i += 1
        if(i == height):
            break
    
    return (width, height, cells)

def read_tasks_from_movingai_file(path):
    tasks = []
    task_file = open(path)
    for l in task_file:
        new_task = l.split()[4:]
        if len(new_task) != 0:
            tasks.append(list(map(float,new_task)))
    #возвращает числа: координаты начала и конца, длину пути
    return np.array(tasks, int)

In [4]:
class HighNode:
    '''
    HighNode class represents a high-level search node

    - vertexCons: vertex constraints of the node
    - edgeCons: edge constraints of the node
    - sol: solution of the node
    - g: cost of the solution of the node 
    - h: h-value of the node
    - F: f-value of the node
    - parent: pointer to the parent-node 

    '''

    def __init__(self, vertexCons={}, edgeCons={}, sol={}, g=0, h=0, F=None, parent=None, k=0):
        self.vertexCons = vertexCons
        self.edgeCons = edgeCons
        self.sol = sol
        self.g = g
        self.h = h
        self.k = k
        if F is None:
            self.F = g + h
        else:
            self.F = F        
        self.parent = parent
    
    
    def __eq__(self, other):
        return (self.vertexCons == other.vertexCons) and (self.edgeCons == other.edgeCons) and \
               (self.sol == other.sol)
    
    def __lt__(self, other):
        return self.F < other.F or ((self.F == other.F) and (self.h < other.h)) \
        or ((self.F == other.F) and (self.h == other.h))

In [5]:
class LowNode:
    '''
    LowNode class represents a low-level search node

    - i, j: coordinates of corresponding grid element
    - g: g-value of the node 
    - h: h-value of the node
    - F: f-value of the node
    - parent: pointer to the parent-node 
    '''

    def __init__(self, coord, g=0, h=0, F=None, parent=None):
        self.i = coord[0]
        self.j = coord[1]
        self.g = g
        self.h = h
        if F is None:
            self.F = g + h
        else:
            self.F = F        
        self.parent = parent
    
    
    def __eq__(self, other):
        return (self.i == other.i) and (self.j == other.j) and (self.g == other.g)
    
    def __lt__(self, other):
        return self.F < other.F or ((self.F == other.F) and (self.h < other.h)) \
        or ((self.F == other.F) and (self.h == other.h))

In [6]:
class OpenHigh:
    
    def __init__(self):
        self.heap = []
    
    
    def __iter__(self):
        return iter(self.heap)
    
    
    def AddNode(self, node : HighNode):
        heappush(self.heap, node)

        
    def GetBestNode(self):        
        best = heappop(self.heap)
        
        return best

In [7]:
class OpenLow:
    
    def __init__(self):
        self.heap = []
        self.elements = {}
    
    def __iter__(self):
        return iter(self.elements.values())
    
    def __len__(self):
        return len(self.elements)

    def isEmpty(self):
        if len(self.elements) != 0:
            return False
        return True

    def AddNode(self, node : LowNode, *args):
        if self.elements.get((node.i, node.j, node.g)) is None or node < self.elements[(node.i, node.j, node.g)]:
            self.elements[(node.i, node.j, node.g)] = node
            heappush(self.heap, node)
        return

    def GetBestNode(self, CLOSED, *args):
        best = heappop(self.heap)    
        while CLOSED.WasExpanded(best):
            best = heappop(self.heap)
        del self.elements[(best.i, best.j, best.g)]
        return best

In [8]:
class ClosedLow:
    
    def __init__(self):
        self.elements = {}


    def __iter__(self):
        return iter(self.elements.values())
    

    def __len__(self):
        return len(self.elements)


    def AddNode(self, node : LowNode):
        self.elements[(node.i, node.j, node.g)] = node


    def WasExpanded(self, node : LowNode):
        return not self.elements.get((node.i, node.j, node.g)) is None

In [9]:
def ManhattanDistance(i1, j1, i2, j2):
    return abs(i1 - i2) + abs(j1 - j2)

In [10]:
def MakePath(goal):
    '''
    Creates a path by tracing parent pointers from the goal node to the start node
    It also returns path's length.
    '''

    length = goal.g
    current = goal
    path = []
    while current.parent:
        path.append(current)
        current = current.parent
    path.append(current)
    return path[::-1], length

In [11]:
class AstarTimesteps:
    def __init__(self, gridMap, iStart, jStart, iGoal, jGoal, vertexCons, edgeCons):
        self.vertexCons = vertexCons
        self.edgeCons = edgeCons
        self.CLOSED = ClosedLow()
        self.gridMap = gridMap
        self.iStart = iStart
        self.jStart = jStart
        self.iGoal = iGoal
        self.jGoal = jGoal
        self.OPEN = OpenLow()
        self.path = []
        
    def CheckMove(self, i1, j1, i2, j2, t):
        for obs in self.vertexCons:
            if obs[1] == t + 1 and obs[0] == (i2, j2):
                return False
        
        for obs in self.edgeCons:
            if obs[2] == t + 1 and obs[0] == (i1, j1) and obs[1] == (i2, j2):
                return False
                
        return True
    
    def FindPath(self):
        startNode = LowNode(coord=(self.iStart, self.jStart))
        self.OPEN.AddNode(startNode)
    
        while not self.OPEN.isEmpty():
            s = self.OPEN.GetBestNode(self.CLOSED) 
            self.CLOSED.AddNode(s)       
            if s.i == self.iGoal and s.j == self.jGoal:
                return (True, s, self.OPEN, self.CLOSED)
            for nbr in self.gridMap.GetNeighbors(s.i, s.j):
                if self.CheckMove(s.i, s.j, nbr[0], nbr[1], s.g) and \
                not self.CLOSED.WasExpanded(LowNode(coord=nbr, g=s.g + 1)):
                    nbrNode = LowNode(coord=nbr, g=s.g + 1, h=ManhattanDistance(nbr[0], nbr[1], self.iGoal, self.jGoal), \
                                      parent=s)
                    self.OPEN.AddNode(nbrNode)
        return (False, None, self.OPEN, self.CLOSED)

In [13]:
def BP_CBS(gridMap, Starts, Goals, subset=[], vertexCons=[], edgeCons=[]): # subset, vertex/edge- Cons - нужны только при вызове CBS в качестве нижнего уровня MACBS
    tic = time.perf_counter() # начало работы функции
    
    gen = 0
    exp = 0
    
    root = HighNode(vertexCons={}, edgeCons={}, sol={}, k=gen)
    OPEN = OpenHigh()
    agents = list(range(len(Starts)))
    if len(subset) > 0:
        agents = subset.copy()
    
    for vc in vertexCons:
        # перебираем агентов из подмножества
        for a in vc[0]:
            if a in root.vertexCons:
                root.vertexCons[a].append((vc[1], vc[2]))
            else:
                root.vertexCons[a] = [(vc[1], vc[2])]
                
    for ec in edgeCons:
        # перебираем агентов из подмножества
        for a in ec[0]:
            if a in root.edgeCons:
                root.edgeCons[a].append((ec[1], ec[2], ec[3]))
            else:
                root.edgeCons[a] = [(ec[1], ec[2], ec[3])]
                
    for a in agents:
        VC = []
        EC = []
        if a in root.vertexCons:
            VC = root.vertexCons[a]
        if a in root.edgeCons:
            EC = root.edgeCons[a]
            
        planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], VC, EC)
        res = planner.FindPath()
        if res[0]:
            path = MakePath(res[1])
            root.sol[a], _ = path
        else:
            return (False, None, gen, exp)
    
    root.g = sum([len(path) for path in root.sol.values()])
    OPEN.AddNode(root)
    gen += 1
    
    toc = time.perf_counter()
    
    while toc - tic < 120:
        s = OPEN.GetBestNode()
        exp += 1
        
        newVertexCons = []
        newEdgeCons = []
        
        for i, a in enumerate(agents):
            for b in agents[i + 1 :]:
                for step in range(min(len(s.sol[a]), len(s.sol[b]))):
                    if s.sol[a][step].i == s.sol[b][step].i and s.sol[a][step].j == s.sol[b][step].j:
                        newVertexCons.append((a, b, (s.sol[a][step].i, s.sol[a][step].j), step))
                    if step + 1 < min(len(s.sol[a]), len(s.sol[b])) and \
                       s.sol[a][step].i == s.sol[b][step + 1].i and s.sol[a][step].j == s.sol[b][step + 1].j and \
                       s.sol[a][step + 1].i == s.sol[b][step].i and s.sol[a][step + 1].i == s.sol[b][step].i:
                        newEdgeCons.append((
                            a,
                            b,
                            (s.sol[a][step].i, s.sol[a][step].j),
                            (s.sol[a][step + 1].i, s.sol[a][step + 1].j),
                            step,
                        ))
                        
        if len(newVertexCons) == 0 and len(newEdgeCons) == 0:
            return (True, s, gen, exp)
        
        # Сейчас сначала разрешаются вершинные конфликты, потом реберные
        if len(newVertexCons) > 0:
            a, b, (i, j), t = newVertexCons[0]
            
            FlagChildA = False
            FlagChildB = False
            
            # Разбиваем CT на ноды A и B, разрешая вершинный конфликт 
            
            vertexCons_tmp_A = s.vertexCons.copy()
            edgeCons_tmp_A=s.edgeCons.copy()
            sol_tmp_A=s.sol.copy()
            
            if a in vertexCons_tmp_A:
                vertexCons_tmp_A[a].append(((i, j), t))   
            else:
                vertexCons_tmp_A[a] = [((i, j), t)]
            
            #A = HighNode(vertexCons=tmp, edgeCons=s.edgeCons.copy(), sol=s.sol.copy(), parent=s, k=gen)
            
            
            ec = []
            if a in edgeCons_tmp_A:
                ec = edgeCons_tmp_A[a]
            
            planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], vertexCons_tmp_A[a], ec)
            res = planner.FindPath()
            if res[0]:
                path = MakePath(res[1])
                sol_tmp_A[a], _ = path
                g = sum([len(path) for path in sol_tmp_A.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if  g == s.g:
                    FlagChildA = True 
                
                
            vertexCons_tmp_B = s.vertexCons.copy()
            edgeCons_tmp_B=s.edgeCons.copy()
            sol_tmp_B=s.sol.copy()
            
            if b in vertexCons_tmp_B:
                vertexCons_tmp_B[b].append(((i, j), t))   
            else:
                vertexCons_tmp_B[b] = [((i, j), t)]
                            
            ec = []
            if b in edgeCons_tmp_B:
                ec = edgeCons_tmp_B[b]
            
            planner = AstarTimesteps(gridMap, Starts[b][0], Starts[b][1], Goals[b][0], Goals[b][1], vertexCons_tmp_B[b], ec)
            res = planner.FindPath()
            if res[0]:
                path = MakePath(res[1])
                sol_tmp_B[b], _ = path
                g = sum([len(path) for path in sol_tmp_B.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if g == s.g:
                    FlagChildB = True 
                
            if FlagChildA:
                s.vertexCons=vertexCons_tmp_A
                s.edgeCons=edgeCons_tmp_A
                s.sol=sol_tmp_A
            elif FlagChildB:
                s.vertexCons=tmp_B
                s.edgeCons=edgeCons_tmp_B
                s.sol=sol_tmp_B
            else:
                A = HighNode(
                    vertexCons=vertexCons_tmp_A,
                    edgeCons=edgeCons_tmp_A,
                    sol=sol_tmp_A,
                    parent=s,
                    k=gen,
                )
                gen+=1
                B = HighNode(
                    vertexCons=vertexCons_tmp_B,
                    edgeCons=edgeCons_tmp_B,
                    sol=sol_tmp_B,
                    parent=s,
                    k=gen,
                )
                gen+=1
                OPEN.AddNode(A)
                OPEN.AddNode(B)
                
                      

                
        elif len(newEdgeCons) > 0:
            a, b, (i1, j1), (i2, j2), t = newEdgeCons[0]
            
            FlagChildA = False
            FlagChildB = False
            
            # Разбиваем CT на ноды A и B, разрешая реберный конфликт 
            
            
            vertexCons_tmp_A = s.vertexCons.copy()
            edgeCons_tmp_A=s.edgeCons.copy()
            sol_tmp_A=s.sol.copy()
            
            if a in edgeCons_tmp_A:
                edgeCons_tmp_A[a].append(((i1, j1), (i2, j2), t))   
            else:
                edgeCons_tmp_A[a] = [((i1, j1), (i2, j2), t)]
            
            
            vc = []
            if a in vertexCons_tmp_A:
                vc = vertexCons_tmp_A[a]
            
            planner = AstarTimesteps(gridMap, Starts[a][0], Starts[a][1], Goals[a][0], Goals[a][1], vc, vertexCons_tmp_A[a])
            res = planner.FindPath()
            if res[0]:
                path = MakePath(res[1])
                sol_tmp_A[a], _ = path
                g = sum([len(path) for path in sol_tmp_A.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if  g == s.g:
                    FlagChildA = True 
                
                
            vertexCons_tmp_B = s.vertexCons.copy()
            edgeCons_tmp_B=s.edgeCons.copy()
            sol_tmp_B=s.sol.copy()
            
            if b in edgeCons_tmp_B:
                edgeCons_tmp_B[b].append(((i1, j1), (i2, j2), t))   
            else:
                edgeCons_tmp_B[b] = [((i1, j1), (i2, j2), t)]
            
            vc = []
            if b in vertexCons_tmp_B:
                vc = vertexCons_tmp_B[b]
            
            planner = AstarTimesteps(gridMap, Starts[b][0], Starts[b][1], Goals[b][0], Goals[b][1], vc, edgeCons_tmp_B[b])
            res = planner.FindPath()
            if res[0]:
                path = MakePath(res[1])
                sol_tmp_B[b], _ = path
                g = sum([len(path) for path in sol_tmp_B.values()]) # SIC, можно использовать другой cost; добавить подсчет h, F
                if  g == s.g:
                    FlagChildB = True 
                    
            if FlagChildA:
                s.vertexCons=vertexCons_tmp_A
                s.edgeCons=edgeCons_tmp_A
                s.sol=sol_tmp_A
            elif FlagChildB:
                s.vertexCons=tmp_B
                s.edgeCons=edgeCons_tmp_B
                s.sol=sol_tmp_B
            else:
                A = HighNode(
                    vertexCons=vertexCons_tmp_A,
                    edgeCons=edgeCons_tmp_A,
                    sol=sol_tmp_A,
                    parent=s,
                    k=gen,
                )
                gen+=1
                B = HighNode(
                    vertexCons=vertexCons_tmp_B,
                    edgeCons=edgeCons_tmp_B,
                    sol=sol_tmp_B,
                    parent=s,
                    k=gen,
                )
                gen+=1
                OPEN.AddNode(A)
                OPEN.AddNode(B)
                
        toc = time.perf_counter()
    
    return (False, None, gen, exp)